# Trie (Prefix Tree)

A Trie is a tree-like data structure optimized for storing and querying keys that are sequences of tokens—most commonly strings as sequences of characters. Each edge represents one token, and a path from the root to a node spells out a prefix; a special end marker indicates a complete key.

Key properties and benefits:
- Fast prefix queries: efficient for autocomplete, spell-check suggestions, IP routing, and dictionary operations.
- Deterministic behavior: no hashing; lookups follow the key's tokens.
- Space–time trade-off: often uses more memory than hash tables due to many nodes but shines when many keys share prefixes.

Typical operations (n = length of key, p = length of prefix):
- Insert: O(n)
- Search (exact): O(n)
- Starts-with (prefix existence): O(p)
- Enumerate keys by prefix: O(p + k) where k is total length of reported keys

Common use cases:
- Autocomplete and search-as-you-type
- Spell checkers and word games (e.g., Boggle)
- Longest-prefix match (e.g., routing tables)
- Dictionary implementation with ordered traversal by prefix

In [2]:
package main

import (
    "fmt"
    "strings"
)

// TrieNode represents a node in the trie
type TrieNode[T comparable] struct {
    children map[T]*TrieNode[T]
    terminal bool
    value    interface{}
}

// NewTrieNode creates a new trie node
func NewTrieNode[T comparable]() *TrieNode[T] {
    return &TrieNode[T]{
        children: make(map[T]*TrieNode[T]),
        terminal: false,
        value:    nil,
    }
}

// Trie represents a generic trie over sequences of tokens
type Trie[T comparable] struct {
    root *TrieNode[T]
    size int
}

// NewTrie creates a new trie
func NewTrie[T comparable]() *Trie[T] {
    return &Trie[T]{
        root: NewTrieNode[T](),
        size: 0,
    }
}

// Len returns the number of keys in the trie
func (t *Trie[T]) Len() int {
    return t.size
}

// Insert adds a key with optional value
func (t *Trie[T]) Insert(key []T, value interface{}) {
    node := t.root
    for _, token := range key {
        if _, exists := node.children[token]; !exists {
            node.children[token] = NewTrieNode[T]()
        }
        node = node.children[token]
    }
    if !node.terminal {
        node.terminal = true
        t.size++
    }
    node.value = value
}

// Contains checks if a key exists as a complete entry
func (t *Trie[T]) Contains(key []T) bool {
    node := t.traverse(key)
    return node != nil && node.terminal
}

// Get returns the value associated with a key
func (t *Trie[T]) Get(key []T) (interface{}, bool) {
    node := t.traverse(key)
    if node != nil && node.terminal {
        return node.value, true
    }
    return nil, false
}

// Delete removes a key if it exists
func (t *Trie[T]) Delete(key []T) bool {
    return t.deleteHelper(t.root, key, 0)
}

func (t *Trie[T]) deleteHelper(node *TrieNode[T], key []T, depth int) bool {
    if depth == len(key) {
        // We've reached the end of the key
        if !node.terminal {
            return false // Key doesn't exist
        }
        node.terminal = false
        node.value = nil
        t.size--
        // Return true if node has no children (can be deleted)
        return len(node.children) == 0
    }

    token := key[depth]
    child, exists := node.children[token]
    if !exists {
        return false // Key doesn't exist
    }

    shouldDeleteChild := t.deleteHelper(child, key, depth+1)
    if shouldDeleteChild {
        delete(node.children, token)
    }

    // Return true if current node can be deleted
    // (no children, not terminal)
    return len(node.children) == 0 && !node.terminal
}

// StartsWith checks if any key exists with given prefix
func (t *Trie[T]) StartsWith(prefix []T) bool {
    node := t.traverse(prefix)
    return node != nil
}

// KeysWithPrefix returns all keys that start with prefix
func (t *Trie[T]) KeysWithPrefix(prefix []T) [][]T {
    node := t.traverse(prefix)
    if node == nil {
        return [][]T{}
    }
    
    var results [][]T
    var path []T
    path = append(path, prefix...)
    t.dfsKeys(node, path, &results)
    return results
}

// LongestPrefixOf returns the longest key that is a prefix of query
func (t *Trie[T]) LongestPrefixOf(query []T) []T {
    node := t.root
    bestEnd := -1
    
    for i, token := range query {
        child, exists := node.children[token]
        if !exists {
            break
        }
        node = child
        if node.terminal {
            bestEnd = i
        }
    }
    
    if bestEnd >= 0 {
        return query[:bestEnd+1]
    }
    return []T{}
}

// traverse follows a path in the trie
func (t *Trie[T]) traverse(key []T) *TrieNode[T] {
    node := t.root
    for _, token := range key {
        child, exists := node.children[token]
        if !exists {
            return nil
        }
        node = child
    }
    return node
}

// dfsKeys performs DFS to collect all keys from a node
func (t *Trie[T]) dfsKeys(node *TrieNode[T], path []T, results *[][]T) {
    if node.terminal {
        // Make a copy of the path
        key := make([]T, len(path))
        copy(key, path)
        *results = append(*results, key)
    }
    
    for token, child := range node.children {
        path = append(path, token)
        t.dfsKeys(child, path, results)
        path = path[:len(path)-1]
    }
}

// CharTrie is a convenience wrapper for string tries
type CharTrie struct {
    trie *Trie[rune]
}

// NewCharTrie creates a new character trie
func NewCharTrie() *CharTrie {
    return &CharTrie{
        trie: NewTrie[rune](),
    }
}

// InsertWord inserts a word
func (ct *CharTrie) InsertWord(word string, value interface{}) {
    runes := []rune(word)
    ct.trie.Insert(runes, value)
}

// ContainsWord checks if a word exists
func (ct *CharTrie) ContainsWord(word string) bool {
    runes := []rune(word)
    return ct.trie.Contains(runes)
}

// WordsWithPrefix returns all words with given prefix
func (ct *CharTrie) WordsWithPrefix(prefix string) []string {
    prefixRunes := []rune(prefix)
    keyRunes := ct.trie.KeysWithPrefix(prefixRunes)
    
    words := make([]string, len(keyRunes))
    for i, keyRune := range keyRunes {
        words[i] = string(keyRune)
    }
    return words
}

// LongestPrefixOfWord returns the longest word that is a prefix of query
func (ct *CharTrie) LongestPrefixOfWord(query string) string {
    queryRunes := []rune(query)
    resultRunes := ct.trie.LongestPrefixOf(queryRunes)
    return string(resultRunes)
}

// Delete removes a word
func (ct *CharTrie) Delete(word string) bool {
    runes := []rune(word)
    return ct.trie.Delete(runes)
}

// Len returns the number of words
func (ct *CharTrie) Len() int {
    return ct.trie.Len()
}

## Examples

Below are quick examples showing trie usage for autocomplete and prefix matching.

In [3]:
%%
// Character trie demo
ct := NewCharTrie()
words := []string{"to", "tea", "ted", "ten", "in", "inn"}
for _, w := range words {
    ct.InsertWord(w, nil)
}

fmt.Println("Size:", ct.Len())
fmt.Println("Contains 'tea':", ct.ContainsWord("tea"))
fmt.Println("Contains 'te':", ct.ContainsWord("te"))
fmt.Println("Starts with 'te':", ct.trie.StartsWith([]rune("te")))
fmt.Println("Words with prefix 'te':", ct.WordsWithPrefix("te"))
fmt.Println("Longest prefix of 'tendril':", ct.LongestPrefixOfWord("tendril"))

// Delete test
fmt.Println("Delete 'ted':", ct.Delete("ted"))
fmt.Println("Contains 'ted':", ct.ContainsWord("ted"))
fmt.Println("Words with prefix 'te':", ct.WordsWithPrefix("te"))

Size: 6
Contains 'tea': true
Contains 'te': false
Starts with 'te': true
Words with prefix 'te': [ted ten tea]
Longest prefix of 'tendril': ten
Delete 'ted': false
Contains 'ted': false
Words with prefix 'te': [tea ten]
